# 05: Perform LCIA

In this script, we perform LCIA on the whole selection of technologies.

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import brightway2 as bw

import time
import csv

In [3]:
bw.projects.set_current(BW_PROJECTNAME)
output_fp = "../output/" + BW_PROJECTNAME

### Get mapping for the technology selection

In [4]:
df = pd.read_csv("../mappings/technology_selection_energy_provision_v4.csv")
long2short = dict(zip(list(df["ecoinvent name"]), list(df["short name"])))

### Search for activities

In [5]:
def get_dbname(scen, year):
    matches = [db for db in bw.databases if "{}_{}".format(scen, year) in db]

    return matches[0]

In [6]:
activities = {}
subset_names = list(df["ecoinvent name"].unique())
for scen in SCENARIOS:
    activities[scen] = {}
    for year in YEARS:
        dbname = get_dbname(scen, year)
        db = bw.Database(dbname)
        db_subset = [act for act in db if act["name"] in subset_names]
        activities[scen][year] = {}
        for sector in df["sector"].unique():
            sel = df[df["sector"] == sector]
            all_matches = []
            for name, product in zip(list(sel["ecoinvent name"]), list(sel["reference product"])):
                matches = [act for act in db_subset if act["name"] == name and act["reference product"] == product]
                print("\t{} activities found for {}, {} in database {}".format(len(matches), name, product, dbname))
                all_matches += matches
            print("{} activities found in {} in database {}".format(len(all_matches), sector, dbname))
            activities[scen][year][sector] = all_matches

	14 activities found for electricity production, at biomass-fired IGCC power plant, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at biomass-fired IGCC power plant, pre, pipeline 200km, storage 1000m, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coal-fired IGCC power plant, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coal-fired power plant, post, pipeline 200km, storage 1000m, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coa

### Get needed LCIA methods

In [7]:
df2 = pd.read_csv("../data/mfs_all_factors.csv").dropna(subset="LCIA method")

In [8]:
needed_methods = list(df2["LCIA method"].unique())
methods = [m for m in list(bw.methods) if ", ".join(list(m)) in needed_methods]

In [9]:
cc_methods = [
    ("IPCC 2021", "climate change: biogenic", "GWP 100a"),
    ("IPCC 2021", "climate change: land use", "GWP 100a"),
    ("IPCC 2021", "climate change: fossil", "GWP 100a")
]
methods = methods + cc_methods

In [10]:
print("{} methods needed, {} methods found".format(len(needed_methods)+len(cc_methods), len(methods)))

73 methods needed, 73 methods found


### Set up file for LCIA output

In [11]:
# open file for writing data
with open(output_fp + "/impacts_energy_provision.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    row = ["short name", "ecoinvent name", "sector",
           "location", "unit", "amount", "product",
           "scenario", "year", "production volume"
           ]
    row += [", ".join(list(m)) for m in methods]
    writer.writerow(row)

In [12]:
def get_production_volume(act):
    p = ""
    for exc in act.exchanges():
        if exc["type"] == "production":
            if "production volume" in exc:
                p = exc["production volume"]

    return p

In [13]:
csvfile = open(output_fp + "/impacts_energy_provision.csv", "a", newline='')
writer = csv.writer(csvfile, delimiter=',')

for scen in SCENARIOS:
    for year in YEARS:
        act_list = []
        sec_list = []
        for sector in df["sector"].unique():
            act_list += activities[scen][year][sector]
            sec_list += len(activities[scen][year][sector]) * [sector,]
        functional_units = [{act: 1} for act in act_list]

        new_setup = {"inv": functional_units, "ia": methods}
        setup_name = "energy_provision_{}_{}".format(scen, year)
        bw.calculation_setups[setup_name] = new_setup

        t0 = time.time()
        mlca = bw.MultiLCA(setup_name)
        t1 = time.time()

        print("MLCA {} took {} seconds".format(setup_name, t1-t0))

        for i, act in enumerate(act_list):
            row = [long2short[act["name"]], act["name"], sec_list[i],
                act["location"], act["unit"], str(1), act["reference product"],
                scen, str(year), str(get_production_volume(act))] + [str(score) for score in mlca.results[i,:]]
            writer.writerow(row)

csvfile.close()

MLCA energy_provision_SSP2-NPi_2020 took 153.65388369560242 seconds
MLCA energy_provision_SSP2-NPi_2030 took 195.99251890182495 seconds
MLCA energy_provision_SSP2-NPi_2040 took 184.9328420162201 seconds
MLCA energy_provision_SSP2-NPi_2050 took 187.96825075149536 seconds
MLCA energy_provision_SSP2-PkBudg500_2020 took 187.5757610797882 seconds
MLCA energy_provision_SSP2-PkBudg500_2030 took 186.50765562057495 seconds
MLCA energy_provision_SSP2-PkBudg500_2040 took 193.97261476516724 seconds
MLCA energy_provision_SSP2-PkBudg500_2050 took 187.56320714950562 seconds
